In [1]:
import open3d.ml.torch as ml3d  # just switch to open3d.ml.tf for tf usage
import numpy as np

Using external Open3D-ML in /Users/sanskara/Workspace/Open3D-ML


In [4]:
# Download weights using link from model zoo(collection of weights for all combinations of model and dataset).
# Link : https://github.com/isl-org/Open3D-ML#semantic-segmentation-1
# Randlanet SemanticKITTI[PyTorch] : https://storage.googleapis.com/open3d-releases/model-zoo/randlanet_semantickitti_202201071330utc.pth
ckpt_path = './randlanet_semantickitti_202201071330utc.pth'

In [5]:
# We define dataset (similar to train_ss_using_pytorch tutorial)
dataset = ml3d.datasets.SemanticKITTI(dataset_path='/Users/sanskara/data/SemanticKITTI/', cache_dir='./logs/cache',training_split=['00'], validation_split=['01'], test_split=['01'])


In [10]:
# Initializing the model and pipeline
model = ml3d.models.RandLANet(in_channels=3)
pipeline = ml3d.pipelines.SemanticSegmentation(model)

In [11]:
# Load checkpoint using `load_ckpt` method (restoring weights for inference)
pipeline.load_ckpt(ckpt_path=ckpt_path)

INFO - 2022-04-01 19:58:03,220 - semantic_segmentation - Loading checkpoint ./randlanet_semantickitti_202201071330utc.pth


In [23]:
test_data = dataset.get_split('test')
data = test_data.get_data(0)

INFO - 2022-04-01 20:12:49,159 - semantickitti - Found 1 pointclouds for test


In [26]:
# For inference on custom data, you can convert your pointcloud in this format.
# Dictionary with keys {'point', 'feat', 'label'}
# If you already have the ground truth labels, you can add it to data to get accuracy and IoU.
# otherwise pass labels as `None`.
print(data)


{'point': array([[ 5.2305943e+01,  2.2989707e-02,  1.9779946e+00],
       [ 5.3259735e+01,  1.0695236e-01,  2.0099745e+00],
       [ 5.3284321e+01,  2.7487758e-01,  2.0109341e+00],
       ...,
       [ 3.8249431e+00, -1.4261885e+00, -1.7655631e+00],
       [ 3.8495324e+00, -1.4222100e+00, -1.7755738e+00],
       [ 3.8631279e+00, -1.4142324e+00, -1.7805853e+00]], dtype=float32), 'feat': None, 'label': array([0, 0, 0, ..., 9, 9, 9], dtype=int32)}


In [19]:
# Running inference on test data
results = pipeline.run_inference(data)
# prints per class accuracy and IoU. Last entry is mean accuracy and mean IoU.
# We get several `nan` outputs for missing classes in the input data.

test 0/1: 100%|██████████| 83500/83500 [00:15<00:00, 303.57it/s]  INFO - 2022-04-01 20:09:24,024 - semantic_segmentation - Accuracy : [0.963756553453385, nan, nan, nan, nan, nan, nan, 0.3561643835616438, 0.9540482683792264, 0.9261289480553381, 0.4870199692780338, nan, 0.9735662148070907, 0.966321243523316, 0.825527363917982, 0.8268921095008052, 0.9399638336347197, 0.9401709401709402, 0.62, 0.81496331902354]
INFO - 2022-04-01 20:09:24,025 - semantic_segmentation - IoU : [0.9090518168135885, nan, 0.0, nan, 0.0, nan, 0.0, 0.3561643835616438, 0.9177174127669178, 0.21501727168050422, 0.47786276800180866, 0.0, 0.9568537022802972, 0.10855646100116415, 0.813112967671631, 0.7186843946815955, 0.6830486202365309, 0.843558282208589, 0.543859649122807, 0.4714679831266924]


In [21]:
# Dictionary of predicted labels and predicted probabilities per class
results

{'predict_labels': array([12, 12, 12, ...,  8,  8,  8], dtype=int16),
 'predict_scores': array([[2.1815e-05, 1.1027e-05, 7.1526e-07, ..., 1.4412e-04, 3.7403e-03,
         1.8473e-03],
        [1.3769e-05, 5.7220e-06, 5.3644e-07, ..., 1.2094e-04, 4.3449e-03,
         2.5768e-03],
        [8.1062e-06, 4.7088e-06, 3.5763e-07, ..., 1.0777e-04, 5.9471e-03,
         3.2253e-03],
        ...,
        [0.0000e+00, 0.0000e+00, 0.0000e+00, ..., 0.0000e+00, 0.0000e+00,
         0.0000e+00],
        [0.0000e+00, 0.0000e+00, 0.0000e+00, ..., 0.0000e+00, 0.0000e+00,
         0.0000e+00],
        [0.0000e+00, 0.0000e+00, 0.0000e+00, ..., 0.0000e+00, 0.0000e+00,
         0.0000e+00]], dtype=float16)}

test 0/1: 100%|██████████| 83500/83500 [00:32<00:00, 303.57it/s]